# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys
import boto3
import botocore
import ipywidgets as widgets

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, model_formatter, tools

In [3]:
bedrock_runtime_client, bedrock_control_client = bedrock.get_bedrock_clients()

Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Create the prompt

In [4]:
# Define a simple prompt
user_input = """
Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

## Select the model

In [5]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

## Build the JSON request payload

In [24]:
# modelId = dropdown.value
body = model_formatter.model_body_builder(modelid  = dropdown.value,
                                           user_input  = user_input,                                             
                                           temperature = 0.9,
                                           topP = 0.99,
                                           topK = 250,
                                           maxTokenCount = 512,
                                           stopSequences = [])

In [25]:
print(body)

{"inputText": "Command: \nWrite an email from Bob, Customer Service Manager, to the customer \"John Doe\" \nwho provided negative feedback on the service provided by our customer support \nengineer", "textGenerationConfig": {"maxTokenCount": 512, "stopSequences": [], "temperature": 0.9, "topP": 0.99}}


## Invoke the LLM

In [26]:
try:

    response = bedrock_runtime_client.invoke_model(body = body, modelId = dropdown.value)

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [27]:
print(response)

{'ResponseMetadata': {'RequestId': '41a8b887-7cab-4cbf-b812-e043f7a5caf5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 02 Jul 2024 21:11:12 GMT', 'content-type': 'application/json', 'content-length': '1415', 'connection': 'keep-alive', 'x-amzn-requestid': '41a8b887-7cab-4cbf-b812-e043f7a5caf5', 'x-amzn-bedrock-invocation-latency': '7553', 'x-amzn-bedrock-output-token-count': '254', 'x-amzn-bedrock-input-token-count': '39'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x14ab7ec80>}


## Print the response

In [28]:
print(model_formatter.model_response_parser(dropdown.value, response))

 Betty
Subject: Apology for the negative experience
Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the negative experience you had with our customer support engineer Betty. We appreciate your feedback and take customer satisfaction very seriously.

I understand that your recent interaction with our team left you frustrated, and I want to assure you that we are committed to resolving this issue. We have conducted an internal investigation and identified areas where we can improve our service.

Please know that your feedback has been shared with our management team, and we will take steps to ensure that this type of situation does not occur again. We value your business and want to provide you with the best possible experience.

In order to compensate for the inconvenience you faced, we would like to offer you a discount on your next purchase with us. We believe that this gesture demonstrates our commitment to making things right and to

## Streaming the response

In [29]:
output = []
try:
    
    response = bedrock_runtime_client.invoke_model_with_response_stream(body=body, modelId=modelId)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

NameError: name 'modelId' is not defined

### Complete output

In [28]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**


Subject: Apology for the inconvenience caused
Dear John Doe,

I am writing to express my deepest apologies for the inconvenience caused by the recent interaction with our customer support engineer. I understand that you had a frustrating experience, and I want to assure you that we take your feedback seriously.

I have thoroughly investigated the matter and discovered that there was a misunderstanding on our part. Our engineer was not trained properly to handle your specific issue, which led to confusion and delays in resolving your problem.

I want to assure you that we have taken steps to rectify the situation. We have provided additional training to our engineering team to ensure that they are equipped to handle a wide range of customer inquiries and issues. We have also implemented a new ticketing system to improve communication and ensure that your concerns are addressed promptly.

Please know that we value your business and your satisfaction is our top pri